# **ETL**

Se descarga la información de la página proporcionada https://datosabiertos.enacom.gob.ar/dashboards/20000/acceso-a-internet/ mediante su descarga en formato xsml, debido a que fue el tipo de archivo que concedió los mejores resultados posibles para la visualización de los datos.

Luego de haber realizado un análisis a grosso modo se decidió realizar un análisis de la cobertura de Internet a nivel nacional en Argentina. Dicho motivo se trabajará con aquellas tablas que aportan datos relevantes para el mismo. 

In [4]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Dataset de Internet

 -- DF: Acceso a Internet Fijo por rangos de velocidad de bajada y provincia

In [5]:
internet_rango_velocidades_provincia = pd.read_excel('./Datasets en bruto/Internet/Internet_Accesos-por-velocidad.xlsx',sheet_name='Accesos por velocidad',dtype=str)
internet_rango_velocidades_provincia

,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
0,2022,4,Buenos Aires,28934,23347,259866,281550,244555,98670,3760109,97634,4794665
1,2022,4,Capital Federal,516,5096,31320,64653,36336,8116,1299117,106772,1551926
2,2022,4,Catamarca,71,315,2716,5028,4638,3122,52739,2280,70909
3,2022,4,Chaco,461,771,16067,17846,6704,3039,94866,3909,143663
4,2022,4,Chubut,109,1614,46457,31007,34728,15770,19048,20026,168759
...,...,...,...,...,...,...,...,...,...,...,...,...
859,2014,1,Santa Cruz,161,1625,24972,1,1,0,0,0,26760
860,2014,1,Santa Fe,8456,124468,345225,20328,6845,23,668,0,506013
861,2014,1,Santiago Del Estero,1234,10531,22817,2422,109,0,0,0,37113
862,2014,1,Tierra Del Fuego,12,607,30902,6,0,0,0,0,31527


In [6]:
# Se renombran las columnas
internet_rango_velocidades_provincia = internet_rango_velocidades_provincia.rename(columns={'HASTA 512 kbps': 'Hasta 512kbps'})
internet_rango_velocidades_provincia = internet_rango_velocidades_provincia.rename(columns={'OTROS': 'Otros'})
internet_rango_velocidades_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Año                  864 non-null    object
 1   Trimestre            864 non-null    object
 2   Provincia            864 non-null    object
 3   Hasta 512kbps        864 non-null    object
 4   + 512 Kbps - 1 Mbps  864 non-null    object
 5   + 1 Mbps - 6 Mbps    864 non-null    object
 6   + 6 Mbps - 10 Mbps   864 non-null    object
 7   + 10 Mbps - 20 Mbps  864 non-null    object
 8   + 20 Mbps - 30 Mbps  864 non-null    object
 9   + 30 Mbps            864 non-null    object
 10  Otros                858 non-null    object
 11  Total                864 non-null    object
dtypes: object(12)
memory usage: 81.1+ KB


In [7]:
# Observamos los valores faltantes en la columna "Otros"
internet_rango_velocidades_provincia[internet_rango_velocidades_provincia['Otros'].isnull()]

,Año,Trimestre,Provincia,Hasta 512kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,Otros,Total
337,2019,2,Capital Federal,74,1027,281370,124087,239691,296155,475991,NaN,1418395
347,2019,2,La Rioja,9,11,39819,4297,4857,13,1298,NaN,50304
348,2019,2,Mendoza,20,2522,205165,2971,10092,1256,234,NaN,222260
355,2019,2,Santa Cruz,25,84,24861,3806,1202,28,4,NaN,30010
358,2019,2,Tierra Del Fuego,6,123,41824,4241,1573,96,32,NaN,47895
359,2019,2,Tucumán,16,157,78891,44296,24291,628,18394,NaN,166673


In [8]:
# A las columnas se les dará su tipo adecuado
internet_rango_velocidades_provincia['Año'] = internet_rango_velocidades_provincia['Año'].astype(int)
internet_rango_velocidades_provincia['Trimestre'] = internet_rango_velocidades_provincia['Trimestre'].astype(int)
columnas_convertir = ['Hasta 512kbps','+ 512 Kbps - 1 Mbps','+ 1 Mbps - 6 Mbps','+ 6 Mbps - 10 Mbps','+ 10 Mbps - 20 Mbps','+ 20 Mbps - 30 Mbps','+ 30 Mbps','Otros','Total']
for columna in columnas_convertir:
    internet_rango_velocidades_provincia[columna] = internet_rango_velocidades_provincia[columna].astype(float)
# Se imputará los valores nulos con los valores tales que al sumarse a las demás columnas de como resultado el "Total"
columnas=['Hasta 512kbps','+ 512 Kbps - 1 Mbps','+ 1 Mbps - 6 Mbps','+ 6 Mbps - 10 Mbps','+ 10 Mbps - 20 Mbps','+ 20 Mbps - 30 Mbps','+ 30 Mbps']
if internet_rango_velocidades_provincia['Otros'].isnull().any():
    suma_columnas = internet_rango_velocidades_provincia[columnas].sum(axis=1)
    internet_rango_velocidades_provincia['Otros'] = [0 if x == y else y - x for x, y in zip(suma_columnas, internet_rango_velocidades_provincia['Total'])]
for columna in columnas_convertir:
    internet_rango_velocidades_provincia[columna] = internet_rango_velocidades_provincia[columna].astype(int)
    
internet_rango_velocidades_provincia.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Año                  864 non-null    int32 
 1   Trimestre            864 non-null    int32 
 2   Provincia            864 non-null    object
 3   Hasta 512kbps        864 non-null    int32 
 4   + 512 Kbps - 1 Mbps  864 non-null    int32 
 5   + 1 Mbps - 6 Mbps    864 non-null    int32 
 6   + 6 Mbps - 10 Mbps   864 non-null    int32 
 7   + 10 Mbps - 20 Mbps  864 non-null    int32 
 8   + 20 Mbps - 30 Mbps  864 non-null    int32 
 9   + 30 Mbps            864 non-null    int32 
 10  Otros                864 non-null    int32 
 11  Total                864 non-null    int32 
dtypes: int32(11), object(1)
memory usage: 44.0+ KB


In [9]:
# Se verifica que no haya filas repetidas
filas_repetidas = internet_rango_velocidades_provincia.duplicated()
if filas_repetidas.any():
    print("Hay filas repetidas")
else:
    print("No hay filas repetidas")

No hay filas repetidas


-- DF: Acceso a Internet fijo por tecnología y provincia

In [10]:
tecnologia_provincia = pd.read_excel('./Datasets en bruto/Internet/Internet_Accesos-por-tecnologia.xlsx',sheet_name='Accesos Por Tecnología',dtype=str)
tecnologia_provincia

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,2022,4,Buenos Aires,325908,2767247,1499149,129910,72451,4794665,NaN,NaN,Buenos Aires,325908,2767247,1499149,129910,72451,4794665
1,2022,4,Capital Federal,138753,1238196,138548,4603,31826,1551926,NaN,NaN,Capital Federal,138753,1238196,138548,4603,31826,1551926
2,2022,4,Catamarca,8952,10998,47337,1379,2243,70909,NaN,NaN,Catamarca,8952,10998,47337,1379,2243,70909
3,2022,4,Chaco,24582,61265,47285,7957,2574,143663,NaN,NaN,Chaco,24582,61265,47285,7957,2574,143663
4,2022,4,Chubut,45450,73923,9815,30721,8850,168759,NaN,NaN,Chubut,45450,73923,9815,30721,8850,168759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,2014,1,Santiago Del Estero,32567,3598,19,915,34,37133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
862,2014,1,Tierra Del Fuego,21618,2837,648,1,2934,28038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
863,2014,1,Tucumán,129717,83,121,13,98,130032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Elimino las columnas demás
columnas =['Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17']
tecnologia_provincia=tecnologia_provincia.drop(columnas,axis=1).reset_index(drop=True)
tecnologia_provincia

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2022,4,Buenos Aires,325908,2767247,1499149,129910,72451,4794665
1,2022,4,Capital Federal,138753,1238196,138548,4603,31826,1551926
2,2022,4,Catamarca,8952,10998,47337,1379,2243,70909
3,2022,4,Chaco,24582,61265,47285,7957,2574,143663
4,2022,4,Chubut,45450,73923,9815,30721,8850,168759
...,...,...,...,...,...,...,...,...,...
861,2014,1,Santiago Del Estero,32567,3598,19,915,34,37133
862,2014,1,Tierra Del Fuego,21618,2837,648,1,2934,28038
863,2014,1,Tucumán,129717,83,121,13,98,130032
864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
tecnologia_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Año           865 non-null    object
 1   Trimestre     865 non-null    object
 2   Provincia     864 non-null    object
 3   ADSL          864 non-null    object
 4   Cablemodem    864 non-null    object
 5   Fibra óptica  864 non-null    object
 6   Wireless      864 non-null    object
 7   Otros         864 non-null    object
 8   Total         864 non-null    object
dtypes: object(9)
memory usage: 61.0+ KB


In [13]:
tecnologia_provincia=tecnologia_provincia.dropna(how='all')
columnas=['ADSL','Cablemodem','Fibra óptica','Wireless','Otros','Total']
tecnologia_provincia.loc[:, columnas] = tecnologia_provincia.loc[:, columnas].astype(float)
tecnologia_provincia

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2022,4,Buenos Aires,325908.0,2767247.0,1499149.0,129910.0,72451.0,4794665.0
1,2022,4,Capital Federal,138753.0,1238196.0,138548.0,4603.0,31826.0,1551926.0
2,2022,4,Catamarca,8952.0,10998.0,47337.0,1379.0,2243.0,70909.0
3,2022,4,Chaco,24582.0,61265.0,47285.0,7957.0,2574.0,143663.0
4,2022,4,Chubut,45450.0,73923.0,9815.0,30721.0,8850.0,168759.0
...,...,...,...,...,...,...,...,...,...
860,2014,1,Santa Fe,322662.0,174296.0,3059.0,5951.0,644.0,506612.0
861,2014,1,Santiago Del Estero,32567.0,3598.0,19.0,915.0,34.0,37133.0
862,2014,1,Tierra Del Fuego,21618.0,2837.0,648.0,1.0,2934.0,28038.0
863,2014,1,Tucumán,129717.0,83.0,121.0,13.0,98.0,130032.0


In [14]:
# Obtener los valores únicos de las columnas Año y Trimestre
valores_unicos_anio = tecnologia_provincia['Año'].unique()
valores_unicos_trimestre = tecnologia_provincia['Trimestre'].unique()
# Mostrar los valores únicos
print("Valores únicos de la columna Año:", valores_unicos_anio)
print("Valores únicos de la columna Trimestre:", valores_unicos_trimestre)

Valores únicos de la columna Año: ['2022' '2021' '2020' '2019' '2019 *' '2018' '2017' '2016' '2015' '2014'
 '*']
Valores únicos de la columna Trimestre: ['4' '3' '2' '1' '3 *' '2 *' '1 *'
 'Los datos provinciales no coinciden a nivel nacional, ya que se rincorporó información que no contien apertuta a nivel geográfico.']


In [15]:
tecnologia_provincia[tecnologia_provincia['Año'] == '2019 *']

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
312,2019 *,3 *,Buenos Aires,1277249.0,2144956.0,425386.0,85572.0,27070.0,3960233.0
313,2019 *,3 *,Capital Federal,300288.0,1080973.0,24900.0,3895.0,31155.0,1441211.0
314,2019 *,3 *,Catamarca,24349.0,5543.0,13.0,244.0,2251.0,32400.0
315,2019 *,3 *,Chaco,50416.0,49156.0,8063.0,5224.0,1592.0,114451.0
316,2019 *,3 *,Chubut,37298.0,27090.0,1917.0,15149.0,7551.0,89005.0
...,...,...,...,...,...,...,...,...,...
379,2019 *,1 *,Santa Cruz,10659.0,4615.0,382.0,6514.0,2735.0,24905.0
380,2019 *,1 *,Santa Fe,340038.0,340344.0,15621.0,23215.0,17860.0,737078.0
381,2019 *,1 *,Santiago Del Estero,25337.0,61547.0,23.0,2847.0,99.0,89853.0
382,2019 *,1 *,Tierra Del Fuego,19789.0,23549.0,1623.0,0.0,1486.0,46447.0


In [16]:
indice=['1 *','2 *','3 *']
tecnologia_provincia[tecnologia_provincia['Trimestre'].isin(indice)]

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
312,2019 *,3 *,Buenos Aires,1277249.0,2144956.0,425386.0,85572.0,27070.0,3960233.0
313,2019 *,3 *,Capital Federal,300288.0,1080973.0,24900.0,3895.0,31155.0,1441211.0
314,2019 *,3 *,Catamarca,24349.0,5543.0,13.0,244.0,2251.0,32400.0
315,2019 *,3 *,Chaco,50416.0,49156.0,8063.0,5224.0,1592.0,114451.0
316,2019 *,3 *,Chubut,37298.0,27090.0,1917.0,15149.0,7551.0,89005.0
...,...,...,...,...,...,...,...,...,...
379,2019 *,1 *,Santa Cruz,10659.0,4615.0,382.0,6514.0,2735.0,24905.0
380,2019 *,1 *,Santa Fe,340038.0,340344.0,15621.0,23215.0,17860.0,737078.0
381,2019 *,1 *,Santiago Del Estero,25337.0,61547.0,23.0,2847.0,99.0,89853.0
382,2019 *,1 *,Tierra Del Fuego,19789.0,23549.0,1623.0,0.0,1486.0,46447.0


In [17]:
# Observamos un valor atípico encontrado
tecnologia_provincia.iloc[864,1]

'Los datos provinciales no coinciden a nivel nacional, ya que se rincorporó información que no contien apertuta a nivel geográfico.'

El "*" indica que no coinciden los datos provinciales con los nacionales por lo que se corroborará esto.

In [18]:
# Se carga el dataset con datos nacionales y se comparará con el dataset provincial
tecnologia_provincia_nacional =pd.read_excel('./Datasets en bruto/Internet/Internet_Accesos-por-tecnologia.xlsx',sheet_name='Totales',dtype=str)
columnas=['ADSL','Cablemodem','Fibra óptica','Wireless','Otros','Total']
tecnologia_provincia_nacional.loc[:, columnas] = tecnologia_provincia_nacional.loc[:, columnas].astype(float)
tecnologia_provincia_nacional.head()

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo
0,2022,4,1315373.0,6037457.0,3060575.0,572969.0,221740.0,11208114.0,Oct-Dic 2022
1,2022,3,1395277.0,6031970.0,2871541.0,557110.0,235230.0,11091128.0,Jul-Sept 2022
2,2022,2,1468333.0,5979214.0,2723285.0,556243.0,231609.0,10958684.0,Abr-Jun 2022
3,2022,1,1533240.0,6073426.0,2219533.0,545814.0,251996.0,10624009.0,Ene-Mar 2022
4,2021,4,1657615.0,5984240.0,2072236.0,523107.0,252596.0,10489794.0,Oct-Dic 2021


In [19]:
# Filtrar el dataframe para el año 2019 y trimestre 3
columnas=['ADSL','Cablemodem','Fibra óptica','Wireless','Otros','Total']
filtro = (tecnologia_provincia['Año'].str.startswith('2019 *')) & (tecnologia_provincia['Trimestre'].str.startswith('3 *'))
subset = tecnologia_provincia.loc[filtro, columnas]
# Calcular las sumas de las columnas
sumas = subset.sum()
# Filtrar el dataframe tecnologia_provincia_nacional para el año y trimestre específico
filtro = (tecnologia_provincia_nacional['Año'] == '2019') & (tecnologia_provincia_nacional['Trimestre'] == '3')
subset_tecnologia_provincia_nacional = tecnologia_provincia_nacional.loc[filtro, columnas]
# Calcular las sumas de las columnas en tecnologia_provincia_nacional
sumas_tecnologia_provincia_nacional = subset_tecnologia_provincia_nacional.sum()
# Crear un nuevo dataframe para comparar los valores
comparacion3 = pd.DataFrame({'Datos Nacionales': sumas_tecnologia_provincia_nacional,
                            'Datos provinciales': sumas})
comparacion3['Iguales'] = comparacion3['Datos Nacionales'] == comparacion3['Datos provinciales']
comparacion3['Diferencia'] = comparacion3['Datos Nacionales'] - comparacion3['Datos provinciales']
# Mostrar el nuevo dataframe de comparación
comparacion3

,Datos Nacionales,Datos provinciales,Iguales,Diferencia
ADSL,2909805.0,3204204.0,False,-294399.0
Cablemodem,4800091.0,4800091.0,True,0.0
Fibra óptica,852221.0,557822.0,False,294399.0
Wireless,338241.0,338241.0,True,0.0
Otros,264326.0,264326.0,True,0.0
Total,9164684.0,9164684.0,True,0.0


In [20]:
# Filtrar el dataframe para el año 2019 y trimestre 2
columnas=['ADSL','Cablemodem','Fibra óptica','Wireless','Otros','Total']
filtro = (tecnologia_provincia['Año'].str.startswith('2019 *')) & (tecnologia_provincia['Trimestre'].str.startswith('2 *'))
subset = tecnologia_provincia.loc[filtro, columnas]
# Calcular las sumas de las columnas
sumas = subset.sum()
# Filtrar el dataframe tecnologia_provincia_nacional para el año y trimestre específico
filtro = (tecnologia_provincia_nacional['Año'] == '2019') & (tecnologia_provincia_nacional['Trimestre'] == '2')
subset_tecnologia_provincia_nacional = tecnologia_provincia_nacional.loc[filtro, columnas]
# Calcular las sumas de las columnas en tecnologia_provincia_nacional
sumas_tecnologia_provincia_nacional = subset_tecnologia_provincia_nacional.sum()
# Crear un nuevo dataframe para comparar los valores
comparacion2 = pd.DataFrame({'Datos Nacionales': sumas_tecnologia_provincia_nacional,
                            'Datos provinciales': sumas})
comparacion2['Iguales'] = comparacion2['Datos Nacionales'] == comparacion2['Datos provinciales']
comparacion2['Diferencia'] = comparacion2['Datos Nacionales'] - comparacion2['Datos provinciales']
# Mostrar el nuevo dataframe de comparación
comparacion2


,Datos Nacionales,Datos provinciales,Iguales,Diferencia
ADSL,2990457.0,3243795.0,False,-253338.0
Cablemodem,4635575.0,4635575.0,True,0.0
Fibra óptica,760794.0,507456.0,False,253338.0
Wireless,308027.0,308027.0,True,0.0
Otros,265328.0,265328.0,True,0.0
Total,8960181.0,8960181.0,True,0.0


In [21]:
# Filtrar el dataframe para el año 2019 y trimestre 1
columnas=['ADSL','Cablemodem','Fibra óptica','Wireless','Otros','Total']
filtro = (tecnologia_provincia['Año'].str.startswith('2019 *')) & (tecnologia_provincia['Trimestre'].str.startswith('1 *'))
subset = tecnologia_provincia.loc[filtro, columnas]
# Calcular las sumas de las columnas
sumas = subset.sum()
# Filtrar el dataframe tecnologia_provincia_nacional para el año y trimestre específico
filtro = (tecnologia_provincia_nacional['Año'] == '2019') & (tecnologia_provincia_nacional['Trimestre'] == '1')
subset_tecnologia_provincia_nacional = tecnologia_provincia_nacional.loc[filtro, columnas]
# Calcular las sumas de las columnas en tecnologia_provincia_nacional
sumas_tecnologia_provincia_nacional = subset_tecnologia_provincia_nacional.sum()
# Crear un nuevo dataframe para comparar los valores
comparacion1 = pd.DataFrame({'Datos Nacionales': sumas_tecnologia_provincia_nacional,
                            'Datos provinciales': sumas})
comparacion1['Iguales'] = comparacion1['Datos Nacionales'] == comparacion1['Datos provinciales']
comparacion1['Diferencia'] = comparacion1['Datos Nacionales'] - comparacion1['Datos provinciales']
# Mostrar el nuevo dataframe de comparación
comparacion1


,Datos Nacionales,Datos provinciales,Iguales,Diferencia
ADSL,3132789.0,3326284.0,False,-193495.0
Cablemodem,4435439.0,4435439.0,True,0.0
Fibra óptica,637099.0,443604.0,False,193495.0
Wireless,276419.0,276419.0,True,0.0
Otros,166389.0,166389.0,True,0.0
Total,8648135.0,8648135.0,True,0.0


Se puede observar que es verdad que los datos no coinciden. Se observa que los valores que no coinciden son los de ADSL y Fibra óptica; sin embargo también se observa que lo que uno pierde el otro lo gana manteniendo el total igual. Es por ello que se ha decidido trabajar con los datos brindados a nivel provincial ya que estos aportan más información al análisis

In [22]:
# Se eliminará el valor atípico y se modificarán los valores con "*"
tecnologia_provincia = tecnologia_provincia.drop(tecnologia_provincia.index[-1]).reset_index(drop=True)
tecnologia_provincia['Trimestre'] = tecnologia_provincia['Trimestre'].apply(lambda x: x.replace(' *', ''))
tecnologia_provincia['Año'] = tecnologia_provincia['Año'].apply(lambda x: x.replace(' *', ''))
# Los paso a su tipo correcto
columnas=['Año','Trimestre','ADSL','Cablemodem','Fibra óptica','Wireless','Otros','Total']
tecnologia_provincia[columnas] = tecnologia_provincia[columnas].astype(int)
# Renombro la columna fibra óptica
tecnologia_provincia = tecnologia_provincia.rename(columns={'Fibra óptica': 'Fibra optica'})

tecnologia_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Año           864 non-null    int32 
 1   Trimestre     864 non-null    int32 
 2   Provincia     864 non-null    object
 3   ADSL          864 non-null    int32 
 4   Cablemodem    864 non-null    int32 
 5   Fibra optica  864 non-null    int32 
 6   Wireless      864 non-null    int32 
 7   Otros         864 non-null    int32 
 8   Total         864 non-null    int32 
dtypes: int32(8), object(1)
memory usage: 33.9+ KB


In [23]:
# Se verifica que no haya filas repetidas
filas_repetidas = tecnologia_provincia.duplicated()
if filas_repetidas.any():
    print("Hay filas repetidas")
else:
    print("No hay filas repetidas")

No hay filas repetidas


-- DF: Accesos a banda ancha y banda angosta por provincia

In [24]:
banda_provincial = pd.read_excel('./Datasets en bruto/Internet/Internet_BAF.xlsx',sheet_name='Dial-BAf',dtype=str)
banda_provincial

,Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total
0,2022,4,Buenos Aires,4788466,6199,4794665
1,2022,4,Capital Federal,1549781,2145,1551926
2,2022,4,Catamarca,70908,1,70909
3,2022,4,Chaco,143658,5,143663
4,2022,4,Chubut,167855,904,168759
...,...,...,...,...,...,...
859,2014,1,Santa Cruz,26304,460,26764
860,2014,1,Santa Fe,506000,612,506612
861,2014,1,Santiago Del Estero,37124,9,37133
862,2014,1,Tierra Del Fuego,27272,766,28038


In [25]:
# Convierto al tipo correcto
columnas= ['Banda ancha fija','Dial up','Total']
banda_provincial[columnas] = banda_provincial[columnas].astype(float)
banda_provincial['Año'] = banda_provincial['Año'].astype(int)
banda_provincial['Trimestre'] = banda_provincial['Trimestre'].astype(int)

banda_provincial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Año               864 non-null    int32  
 1   Trimestre         864 non-null    int32  
 2   Provincia         864 non-null    object 
 3   Banda ancha fija  864 non-null    float64
 4   Dial up           862 non-null    float64
 5   Total             864 non-null    float64
dtypes: float64(3), int32(2), object(1)
memory usage: 33.9+ KB


In [26]:
# Se observan los valores nulos en la columna "Dial up"
banda_provincial[banda_provincial['Dial up'].isnull()]

,Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total
440,2018,2,Formosa,44554.0,NaN,44554.0
445,2018,2,Misiones,106147.0,NaN,106147.0


In [27]:
# Debido a la ausencia de información y que la suma de "Banda ancha fija" y "Dial up" debe dar como suma "Total", a los valores faltantes se les asignará el valor 0
banda_provincial['Dial up']=banda_provincial['Dial up'].fillna(0.0)
columnas= ['Banda ancha fija','Dial up','Total']
banda_provincial[columnas] = banda_provincial[columnas].astype(int)
banda_provincial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Año               864 non-null    int32 
 1   Trimestre         864 non-null    int32 
 2   Provincia         864 non-null    object
 3   Banda ancha fija  864 non-null    int32 
 4   Dial up           864 non-null    int32 
 5   Total             864 non-null    int32 
dtypes: int32(5), object(1)
memory usage: 23.8+ KB


In [28]:
# Se verifica que no haya filas repetidas
filas_repetidas = banda_provincial.duplicated()
if filas_repetidas.any():
    print("Hay filas repetidas")
else:
    print("No hay filas repetidas")

No hay filas repetidas


-- DF: Penetración de Internet fijo (accesos por cada 100 hogares)

In [29]:
internet_100_hogares = pd.read_excel('./Datasets en bruto/Internet/Internet_Penetracion.xlsx',sheet_name='Penetracion-hogares',dtype=str)
internet_100_hogares

,Año,Trimestre,Provincia,Accesos por cada 100 hogares
0,2022,4,Buenos Aires,78.94
1,2022,4,Capital Federal,122.73
2,2022,4,Catamarca,65.77
3,2022,4,Chaco,43.62
4,2022,4,Chubut,85.57
...,...,...,...,...
859,2014,1,Santa Cruz,28.939093063577776
860,2014,1,Santa Fe,47.84766256547095
861,2014,1,Santiago Del Estero,16.309923503967603
862,2014,1,Tierra Del Fuego,63.965925574153744


In [30]:
# Convierto a su tipo correcto:
internet_100_hogares['Accesos por cada 100 hogares'] = internet_100_hogares['Accesos por cada 100 hogares'].astype(float)
internet_100_hogares['Año'] = internet_100_hogares['Año'].astype(int)
internet_100_hogares['Trimestre'] = internet_100_hogares['Trimestre'].astype(int)

internet_100_hogares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           864 non-null    int32  
 1   Trimestre                     864 non-null    int32  
 2   Provincia                     864 non-null    object 
 3   Accesos por cada 100 hogares  864 non-null    float64
dtypes: float64(1), int32(2), object(1)
memory usage: 20.4+ KB


In [31]:
# Se verifica que no haya filas repetidas
filas_repetidas = internet_100_hogares.duplicated()
if filas_repetidas.any():
    print("Hay filas repetidas")
else:
    print("No hay filas repetidas")

No hay filas repetidas


-- DF: Penetración de Internet fijo (accesos por cada 100 habitantes)

In [32]:
internet_100_hab = pd.read_excel('./Datasets en bruto/Internet/Internet_Penetracion.xlsx',sheet_name='Penetración-poblacion',dtype=str)
internet_100_hab

,Año,Trimestre,Provincia,Accesos por cada 100 hab
0,2022,4,Buenos Aires,26.72
1,2022,4,Capital Federal,50.35
2,2022,4,Catamarca,16.73
3,2022,4,Chaco,11.66
4,2022,4,Chubut,26.23
...,...,...,...,...
859,2014,1,Santa Cruz,8.677326900880571
860,2014,1,Santa Fe,15.077800919408592
861,2014,1,Santiago Del Estero,4.058899015580628
862,2014,1,Tierra Del Fuego,19.10492102645171


In [33]:
# Convierto a su tipo correcto:
internet_100_hab['Accesos por cada 100 hab'] = internet_100_hab['Accesos por cada 100 hab'].astype(float)
internet_100_hab['Año'] = internet_100_hab['Año'].astype(int)
internet_100_hab['Trimestre'] = internet_100_hab['Trimestre'].astype(int)

internet_100_hab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Año                       864 non-null    int32  
 1   Trimestre                 864 non-null    int32  
 2   Provincia                 864 non-null    object 
 3   Accesos por cada 100 hab  864 non-null    float64
dtypes: float64(1), int32(2), object(1)
memory usage: 20.4+ KB


In [34]:
# Se verifica que no haya filas repetidas
filas_repetidas = internet_100_hab.duplicated()
if filas_repetidas.any():
    print("Hay filas repetidas")
else:
    print("No hay filas repetidas")

No hay filas repetidas


-- DF: Velocidad media de bajada de Internet fijo por provincia

In [35]:
velocidad_media_provincia = pd.read_excel('./Datasets en bruto/Internet/historico_velocidad_internet.xlsx',sheet_name='Velocidad % por prov',dtype=str)
velocidad_media_provincia

,Año,Trimestre,Provincia,Mbps (Media de bajada)
0,2022,4,Buenos Aires,111
1,2022,4,Capital Federal,182.86
2,2022,4,Catamarca,83.33
3,2022,4,Chaco,104.02
4,2022,4,Chubut,15.92
...,...,...,...,...
859,2014,1,Santa Cruz,3.315991479820628
860,2014,1,Santa Fe,3.1474068452786788
861,2014,1,Santiago Del Estero,2.9494717214992052
862,2014,1,Tierra Del Fuego,3.446887937323564


In [36]:
# Convierto a su tipo correcto:
velocidad_media_provincia['Mbps (Media de bajada)'] = velocidad_media_provincia['Mbps (Media de bajada)'].astype(float)
velocidad_media_provincia['Año'] = velocidad_media_provincia['Año'].astype(int)
velocidad_media_provincia['Trimestre'] = velocidad_media_provincia['Trimestre'].astype(int)

velocidad_media_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Año                     864 non-null    int32  
 1   Trimestre               864 non-null    int32  
 2   Provincia               864 non-null    object 
 3   Mbps (Media de bajada)  864 non-null    float64
dtypes: float64(1), int32(2), object(1)
memory usage: 20.4+ KB


In [37]:
# Se verifica que no haya filas repetidas
filas_repetidas = velocidad_media_provincia.duplicated()
if filas_repetidas.any():
    print("Hay filas repetidas")
else:
    print("No hay filas repetidas")

No hay filas repetidas


-- DF: Accesos a Internet fijo por tecnología y localidad

In [38]:
tecnologia_localidad=pd.read_excel('./Datasets en bruto/Internet/Internet-Accesos-por-tecno_por_loc.xlsx',sheet_name='Accesos_tecnologia_localidad',dtype=str)
tecnologia_localidad

,Provincia,Partido,Localidad,Link Indec,ADSL,CABLEMODEM,DIAL UP,FIBRA OPTICA,OTROS,SATELITAL,WIMAX,WIRELESS,Total general
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,874,4542,0,3,0,852,0,706,6977
1,BUENOS AIRES,25 de Mayo,Del Valle,6854020,181,0,0,0,10,1,0,0,192
2,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,6854040,0,0,0,0,0,0,0,181,181
3,BUENOS AIRES,25 de Mayo,Norberto de la Riestra,6854060,0,782,0,6,167,0,0,327,1282
4,BUENOS AIRES,25 de Mayo,Lucas Monteverde,6854050,0,0,0,0,0,0,0,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3132,TUCUMAN,Tafí Viejo,Villa Mariano Moreno - El Colmenar,90105100,0,31,0,3733,0,0,0,0,3764
3133,TUCUMAN,Trancas,San Pedro de Colalao,90112020,0,0,0,0,0,0,0,313,313
3134,TUCUMAN,Trancas,Villa de Trancas,90112030,126,0,0,0,33,0,0,101,260
3135,TUCUMAN,Yerba Buena,Villa Carmela,90119020,1499,0,0,0,1,0,0,97,1597


In [39]:
# Se elimina la columna "Link Indec" ya que se refiere a la pagina web oficial del INDEC y no aporta datos útiles al análisis
tecnologia_localidad = tecnologia_localidad.drop("Link Indec", axis=1)
# Se renombran las columnas
nuevos_nombres = {'CABLEMODEM': 'Cablemodem','DIAL UP': 'Dial up','FIBRA OPTICA': 'Fibra optica','OTROS': 'Otros','SATELITAL': 'Satelital','WIMAX': 'Wimax','WIRELESS': 'Wireless'}
tecnologia_localidad = tecnologia_localidad.rename(columns=nuevos_nombres)
# Se pasan a sus tipos correspondientes
columnas=['ADSL','Cablemodem','Dial up','Fibra optica','Otros','Satelital','Wimax','Wireless','Total general']
tecnologia_localidad[columnas] = tecnologia_localidad[columnas].astype(int)
# Se corrigen los valores en la columna "Provincia"
tecnologia_localidad['Provincia'] = tecnologia_localidad['Provincia'].str.title()

tecnologia_localidad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3137 entries, 0 to 3136
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Provincia      3137 non-null   object
 1   Partido        3137 non-null   object
 2   Localidad      3137 non-null   object
 3   ADSL           3137 non-null   int32 
 4   Cablemodem     3137 non-null   int32 
 5   Dial up        3137 non-null   int32 
 6   Fibra optica   3137 non-null   int32 
 7   Otros          3137 non-null   int32 
 8   Satelital      3137 non-null   int32 
 9   Wimax          3137 non-null   int32 
 10  Wireless       3137 non-null   int32 
 11  Total general  3137 non-null   int32 
dtypes: int32(9), object(3)
memory usage: 183.9+ KB


In [40]:
# Se verifica que no haya filas repetidas
filas_repetidas = tecnologia_localidad.duplicated()
if filas_repetidas.any():
    print("Hay filas repetidas")
else:
    print("No hay filas repetidas")

No hay filas repetidas


-- DF: Listado de localidades con conectividad a internet

In [41]:
mapa_conectividad=pd.read_excel('./Datasets en bruto/Internet/mapa_conectividad.xlsx',dtype=str)
mapa_conectividad

,Provincia,Partido,Localidad,Población,ADSL,Cablemódem,Dial Up,Fibra óptica,Satelital,Wireless,Telefonía Fija,3G,4G,Link,Latitud,Longitud
0,BUENOS AIRES,9 de Julio,Alfredo Demarchi (Est. Facundo Quiroga),1853,SI,--,--,--,--,--,SI,SI,SI,6588010,-35.2929819536873,-61.4072542128
1,BUENOS AIRES,9 de Julio,Carlos María Naón,497,--,--,--,--,--,--,SI,SI,SI,6588020,-35.239499381663,-60.8251358750153
2,BUENOS AIRES,9 de Julio,Dudignac,2670,--,--,SI,--,--,SI,SI,SI,SI,6588040,-35.6511928511123,-60.7098962799094
3,BUENOS AIRES,9 de Julio,La Aurora (Est. La Niña),531,--,--,--,--,--,--,--,--,SI,6588050,-35.4079467934432,-61.2105668204871
4,BUENOS AIRES,9 de Julio,Manuel B. Gonnet (Est. French),748,SI,--,--,SI,--,--,SI,SI,SI,6588060,-35.5201414507378,-60.9989487976537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4306,BUENOS AIRES,Pilar,Pilar,86158,SI,SI,SI,SI,SI,SI,SI,SI,SI,6638040,-34.4663318396358,-58.9227197887079
4307,BUENOS AIRES,Pilar,Presidente Derqui,50783,SI,SI,--,SI,--,SI,SI,SI,SI,6638040,-34.4928616935261,-58.8416985218723
4308,BUENOS AIRES,Pilar,Roberto De Vicenzo,0,--,--,--,--,--,--,--,SI,SI,6638040,-34.4369652784326,-58.7669941785147
4309,BUENOS AIRES,Pilar,Santa Teresa,5324,--,--,--,--,SI,--,--,SI,SI,6638040,-34.4366914158447,-58.7573470164393


In [42]:
mapa_conectividad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4311 entries, 0 to 4310
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Provincia       4311 non-null   object
 1   Partido         4311 non-null   object
 2   Localidad       4311 non-null   object
 3   Población       4311 non-null   object
 4   ADSL            4311 non-null   object
 5   Cablemódem      4311 non-null   object
 6   Dial Up         4311 non-null   object
 7   Fibra óptica    4311 non-null   object
 8   Satelital       4311 non-null   object
 9   Wireless        4311 non-null   object
 10  Telefonía Fija  4311 non-null   object
 11  3G              4311 non-null   object
 12  4G              4311 non-null   object
 13  Link            4299 non-null   object
 14  Latitud         4311 non-null   object
 15  Longitud        4311 non-null   object
dtypes: object(16)
memory usage: 539.0+ KB


In [43]:
# Se elimina la columna "Link" ya que se refiere a la pagina web oficial del INDEC y no aporta datos útiles al análisis
mapa_conectividad = mapa_conectividad.drop("Link", axis=1)
# Se corrigen los valores en la columna "Provincia"
mapa_conectividad['Provincia'] = mapa_conectividad['Provincia'].str.title()
# Se corrige el tipo de "Población"
mapa_conectividad['Población'] = mapa_conectividad['Población'].astype(int)
# Se renombran las columnas
nuevos_nombres = {'Cablemódem': 'Cablemodem','Población': 'Poblacion','Dial Up': 'Dial up','Fibra óptica':'Fibra optica','Telefonía Fija':'Telefonia fija'}
mapa_conectividad = mapa_conectividad.rename(columns=nuevos_nombres)
# Se cambian los valores "--" y "SI" a valores booleanos:
columnas = ['ADSL', 'Cablemodem', 'Dial up', 'Fibra optica', 'Satelital', 'Wireless', 'Telefonia fija', '3G', '4G']
for columna in columnas:
    mapa_conectividad[columna] = mapa_conectividad[columna].apply(lambda x: True if x == 'SI' else False if x == '--' else x)

In [44]:
mapa_conectividad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4311 entries, 0 to 4310
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Provincia       4311 non-null   object
 1   Partido         4311 non-null   object
 2   Localidad       4311 non-null   object
 3   Poblacion       4311 non-null   int32 
 4   ADSL            4311 non-null   bool  
 5   Cablemodem      4311 non-null   bool  
 6   Dial up         4311 non-null   bool  
 7   Fibra optica    4311 non-null   bool  
 8   Satelital       4311 non-null   bool  
 9   Wireless        4311 non-null   bool  
 10  Telefonia fija  4311 non-null   bool  
 11  3G              4311 non-null   bool  
 12  4G              4311 non-null   bool  
 13  Latitud         4311 non-null   object
 14  Longitud        4311 non-null   object
dtypes: bool(9), int32(1), object(5)
memory usage: 223.3+ KB


-- DF: Ingresos trimestrales por la prestación del servicio de Internet fijo

In [45]:
ingresos_internet=pd.read_excel('./Datasets en bruto/Internet/Internet_Ingresos.xlsx',dtype=str)
ingresos_internet

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2022,4,72443482.9891497,Oct-Dic 2022
1,2022,3,64086784.3305304,Jul-Sept 2022
2,2022,2,60419500.6232803,Abr-Jun 2022
3,2022,1,55219313.0146503,Ene-Mar 2022
4,2021,4,45467887.16501248,Oct-Dic 2021
5,2021,3,42999944.292822465,Jul-Sept 2021
6,2021,2,38239666.63354247,Abr-Jun 2021
7,2021,1,36676371.36239245,Ene-Mar 2021
8,2020,4,33539702.716311768,Oct-Dic 2020
9,2020,3,31997444.530699246,Jul-Sept 2020


In [46]:
# Convierto a su tipo correcto:
ingresos_internet['Ingresos (miles de pesos)'] = ingresos_internet['Ingresos (miles de pesos)'].astype(float)
ingresos_internet['Año'] = ingresos_internet['Año'].astype(int)
ingresos_internet['Trimestre'] = ingresos_internet['Trimestre'].astype(int)
ingresos_internet=ingresos_internet.drop(columns='Periodo',axis=1)

ingresos_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Año                        36 non-null     int32  
 1   Trimestre                  36 non-null     int32  
 2   Ingresos (miles de pesos)  36 non-null     float64
dtypes: float64(1), int32(2)
memory usage: 708.0 bytes


In [47]:
# Se crean los datasets limpios
internet_velocidad=pd.merge(internet_rango_velocidades_provincia,velocidad_media_provincia,on=['Año','Trimestre','Provincia'])
acceso_internet=pd.merge(internet_100_hogares,internet_100_hab,on=['Año','Trimestre','Provincia'])
internet_velocidad.to_csv('./Datasets limpios/internet_velocidad.csv',index=False)
acceso_internet.to_csv('./Datasets limpios/acceso_internet.csv',index=False)
tecnologia_localidad.to_csv('./Datasets limpios/tecnologia_localidad.csv',index=False)
mapa_conectividad.to_csv('./Datasets limpios/mapa_conectividad.csv',index=False)
ingresos_internet.to_csv('./Datasets limpios/ingresos_internet.csv',index=False)
tecnologia_provincia.to_csv('./Datasets limpios/tecnologia_provincia.csv',index=False)
banda_provincial.to_csv('./Datasets limpios/banda_provincial.csv',index=False)

# Dataset de Telefonía fija

-- DF: Penetración de Telefonía fija (accesos por cada 100 hogares)

In [48]:
penetracion_telefonia_fija=pd.read_excel('./Datasets en bruto/Telefonia fija/telefonia_fija.xlsx',sheet_name='Fija_penetracion_prov',dtype=str).sort_values(by=['Año','Trimestre'],ascending=[False,False]).reset_index(drop=True)
penetracion_telefonia_fija

,Año,Trimestre,Provincia,Accesos telefonía fija por cada 100 hab,Accesos telefonía fija por cada 100 hogares
0,2022,4,Buenos Aires,17.763738794105226,53.50257390533205
1,2022,4,Capital Federal,50.80126734907988,122.72706534018253
2,2022,4,Catamarca,6.016207477593805,23.651659885809703
3,2022,4,Chaco,5.348296317660513,20.011183343984328
4,2022,4,Chubut,8.704445533425885,28.398139837487445
...,...,...,...,...,...
859,2014,1,Santa Cruz,13.171295516057151,44.250456978123715
860,2014,1,Santa Fe,22.927498048534865,72.91043546719332
861,2014,1,Santiago del Estero,7.7298672793691665,31.145180043689663
862,2014,1,Tierra del Fuego,22.234563022118046,74.97587427048389


In [49]:
# Convierto a su tipo correcto:
penetracion_telefonia_fija['Accesos telefonía fija por cada 100 hab'] = penetracion_telefonia_fija['Accesos telefonía fija por cada 100 hab'].astype(float)
penetracion_telefonia_fija['Accesos telefonía fija por cada 100 hogares'] = penetracion_telefonia_fija['Accesos telefonía fija por cada 100 hogares'].astype(float)
penetracion_telefonia_fija['Año'] = penetracion_telefonia_fija['Año'].astype(int)
penetracion_telefonia_fija['Trimestre'] = penetracion_telefonia_fija['Trimestre'].astype(int)

penetracion_telefonia_fija.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 5 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Año                                          864 non-null    int32  
 1   Trimestre                                    864 non-null    int32  
 2   Provincia                                    864 non-null    object 
 3   Accesos telefonía fija por cada 100 hab      864 non-null    float64
 4   Accesos telefonía fija por cada 100 hogares  864 non-null    float64
dtypes: float64(2), int32(2), object(1)
memory usage: 27.1+ KB


-- DF: Penetración trimestral de la telefonía fija por provincia

In [50]:
telefonia_tipo_provincia=pd.read_excel('./Datasets en bruto/Telefonia fija/telefonia_fija.xlsx',sheet_name='Fija_prov_tipo',dtype=str)
telefonia_tipo_provincia

,Año,Trimestre,Provincia,Accesos telefonía fija (total),Accesos telefonía fija (hogares),Accesos telefonía fija (comercial),Accesos telefonía fija (gobierno),Accesos telefonía fija (otros)
0,2022,4,Buenos Aires,3128431,2802796,264038,12029,49568
1,2022,4,Capital Federal,1563142,1400437,131928,6010,24767
2,2022,4,Catamarca,25083,22473,2117,96,397
3,2022,4,Chaco,64683,57950,5459,249,1025
4,2022,4,Chubut,54249,48601,4579,209,860
...,...,...,...,...,...,...,...,...
859,2014,1,Santa Cruz,40625,33555,4528,131,2411
860,2014,1,Santa Fe,770361,636284,85867,2491,45719
861,2014,1,Santiago del Estero,70717,58409,7882,229,4197
862,2014,1,Tierra del Fuego,32631,26951,3637,106,1937


In [51]:
# Convierto a su tipo correcto:
columnas= ['Año','Trimestre']
telefonia_tipo_provincia[columnas] = telefonia_tipo_provincia[columnas].astype(int)
columnas=['Accesos telefonía fija (hogares)','Accesos telefonía fija (comercial)','Accesos telefonía fija (gobierno)','Accesos telefonía fija (otros)']
telefonia_tipo_provincia= telefonia_tipo_provincia.drop(columnas, axis=1)
telefonia_tipo_provincia['Accesos telefonía fija (total)'] = telefonia_tipo_provincia['Accesos telefonía fija (total)'].astype(int)

telefonia_tipo_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Año                             864 non-null    int32 
 1   Trimestre                       864 non-null    int32 
 2   Provincia                       864 non-null    object
 3   Accesos telefonía fija (total)  864 non-null    int32 
dtypes: int32(3), object(1)
memory usage: 17.0+ KB


In [52]:
# Se unen los 2 dataframes anteriores
telefonia_provincia = pd.merge(penetracion_telefonia_fija, telefonia_tipo_provincia, on=["Año", "Trimestre", "Provincia"])
telefonia_provincia

,Año,Trimestre,Provincia,Accesos telefonía fija por cada 100 hab,Accesos telefonía fija por cada 100 hogares,Accesos telefonía fija (total)
0,2022,4,Buenos Aires,17.763739,53.502574,3128431
1,2022,4,Capital Federal,50.801267,122.727065,1563142
2,2022,4,Catamarca,6.016207,23.651660,25083
3,2022,4,Chaco,5.348296,20.011183,64683
4,2022,4,Chubut,8.704446,28.398140,54249
...,...,...,...,...,...,...
859,2014,1,Santa Cruz,13.171296,44.250457,40625
860,2014,1,Santa Fe,22.927498,72.910435,770361
861,2014,1,Santiago del Estero,7.729867,31.145180,70717
862,2014,1,Tierra del Fuego,22.234563,74.975874,32631


-- DF: Ingresos trimestrales por la prestación del servicio de telefonía fija

In [53]:
ingresos_telefonia_fija=pd.read_excel('./Datasets en bruto/Telefonia fija/telefonia_fija.xlsx',sheet_name='Fija_ingresos',dtype=str).sort_values(by=['Año','Trimestre'],ascending=[False,False]).reset_index(drop=True)
ingresos_telefonia_fija

,Año,Trimestre,Ingresos (miles de $)
0,2022,4,32356913.919
1,2022,3,29698512.567
2,2022,2,31258182.505
3,2022,1,30583344.001
4,2021,4,40616059.171
5,2021,3,20789995.213
6,2021,2,22178751.89
7,2021,1,21378667.685
8,2020,4,19750303.976
9,2020,3,18162023.887


In [54]:
# Convierto a su tipo correcto:
ingresos_telefonia_fija['Ingresos (miles de $)'] = ingresos_telefonia_fija['Ingresos (miles de $)'].astype(float)
ingresos_telefonia_fija['Año'] = ingresos_telefonia_fija['Año'].astype(int)
ingresos_telefonia_fija['Trimestre'] = ingresos_telefonia_fija['Trimestre'].astype(int)

ingresos_telefonia_fija.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Año                    36 non-null     int32  
 1   Trimestre              36 non-null     int32  
 2   Ingresos (miles de $)  36 non-null     float64
dtypes: float64(1), int32(2)
memory usage: 708.0 bytes


In [55]:
# Creación de datasets limpios
telefonia_provincia.to_csv('./Datasets limpios/acceso_telefonia_fija.csv',index=False)
ingresos_telefonia_fija.to_csv('./Datasets limpios/ingresos_telefonia_fija.csv',index=False)

# Dataset de Telefonía móvil

In [56]:
penetracion_telefonia_movil=pd.read_excel('./Datasets en bruto/Telefonía movil/Telefonia_movil.xlsx',sheet_name='Penetracion',dtype=str).sort_values(by=['Año','Trimestre'],ascending=[False,False]).reset_index(drop=True)
penetracion_telefonia_movil

,Año,Trimestre,Accesos por cada 100 hab,Periodo
0,2022,4,128.65,Oct-Dic 2022
1,2022,3,127.98,Jul-Sept 2022
2,2022,2,127.95,Abr-Jun 2022
3,2022,1,128.37,Ene-Mar 2022
4,2021,4,128.44,Oct-Dic 2021
5,2021,3,126.01,Jul-Sept 2021
6,2021,2,124.26,Abr-Jun 2021
7,2021,1,123.62,Ene-Mar 2021
8,2020,4,120.21,Oct-Dic 2020
9,2020,3,120.18,Jul-Sept 2020


In [57]:
# Convierto a su tipo correcto:
penetracion_telefonia_movil['Accesos por cada 100 hab'] = penetracion_telefonia_movil['Accesos por cada 100 hab'].astype(float)
penetracion_telefonia_movil['Año'] = penetracion_telefonia_movil['Año'].astype(int)
penetracion_telefonia_movil['Trimestre'] = penetracion_telefonia_movil['Trimestre'].astype(int)
penetracion_telefonia_movil = penetracion_telefonia_movil.drop('Periodo',axis=1)

penetracion_telefonia_movil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Año                       40 non-null     int32  
 1   Trimestre                 40 non-null     int32  
 2   Accesos por cada 100 hab  40 non-null     float64
dtypes: float64(1), int32(2)
memory usage: 772.0 bytes


-- DF: Ingresos trimestrales por la prestación del servicio de telefonía móvil

In [58]:
ingresos_telefonia_movil=pd.read_excel('./Datasets en bruto/Telefonía movil/Telefonia_movil.xlsx',sheet_name='Ingresos',dtype=str).sort_values(by=['Año','Trimestre'],ascending=[False,False]).reset_index(drop=True)
ingresos_telefonia_movil

,Año,Trimestre,Ingresos (miles de $),Periodo
0,2022,4,204796575,Oct-Dic 2022
1,2022,3,180356862,Jul-Sept 2022
2,2022,2,153032313,Abr-Jun 2022
3,2022,1,130454676,Ene-Mar 2022
4,2021,4,127389786,Oct-Dic 2021
5,2021,3,117751925,Jul-Sept 2021
6,2021,2,104782020,Abr-Jun 2021
7,2021,1,93112153,Ene-Mar 2021
8,2020,4,83479114,Oct-Dic 2020
9,2020,3,79634755,Jul-Sept 2020


In [59]:
# Convierto a su tipo correcto:
ingresos_telefonia_movil['Ingresos (miles de $)'] = ingresos_telefonia_movil['Ingresos (miles de $)'].astype(int)
ingresos_telefonia_movil['Año'] = ingresos_telefonia_movil['Año'].astype(int)
ingresos_telefonia_movil['Trimestre'] = ingresos_telefonia_movil['Trimestre'].astype(int)
ingresos_telefonia_movil = ingresos_telefonia_movil.drop('Periodo',axis=1)

ingresos_telefonia_movil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   Año                    40 non-null     int32
 1   Trimestre              40 non-null     int32
 2   Ingresos (miles de $)  40 non-null     int32
dtypes: int32(3)
memory usage: 612.0 bytes


-- DF: Accesos telefonía fija

In [60]:
# Se cargan dataframes auxiliares asegurandose que los años y trimestres coincidan
df1=pd.read_excel('./Datasets en bruto/Telefonía movil/Telefonia_movil.xlsx',sheet_name='SMS',dtype=str).sort_values(by=['Año','Trimestre'],ascending=[False,False]).reset_index(drop=True)
df2=pd.read_excel('./Datasets en bruto/Telefonía movil/Telefonia_movil.xlsx',sheet_name='Llamadas salientes',dtype=str).sort_values(by=['Año','Trimestre'],ascending=[False,False]).reset_index(drop=True)
df3=pd.read_excel('./Datasets en bruto/Telefonía movil/Telefonia_movil.xlsx',sheet_name='Accesos',dtype=str).sort_values(by=['Año','Trimestre'],ascending=[False,False]).reset_index(drop=True)
# Se eligen las columnas a unir
columnas=['Año','Trimestre','Número de SMS salientes']
df1_subset=df1[columnas]
df2_subset=df2['Total de llamadas salientes (miles)']
df3_subset=df3['Total de accesos operativos']
# Se arma el dataset final
acceso_telefonia_movil = pd.concat([df1_subset, df2_subset,df3_subset], axis=1)
acceso_telefonia_movil

,Año,Trimestre,Número de SMS salientes,Total de llamadas salientes (miles),Total de accesos operativos
0,2022,4,773515610,5026324.729,59703927
1,2022,3,803389256,5389345.729,59258871
2,2022,2,789033575,5474624.729,59111192
3,2022,1,909954575,5339205.98,59168910
4,2021,4,929840583,5935851.582,59065827
5,2021,3,1109243724,5560488.463,57812509
6,2021,2,748701689,5427525.764,56875562
7,2021,1,1061013967,5585383.163999999,56453033
8,2020,4,1162624348,6007147.805000001,54763900
9,2020,3,1887885018,6256823.481,54619886


In [61]:
columnas=['Año','Trimestre','Total de accesos operativos']
acceso_telefonia_movil[columnas]=acceso_telefonia_movil[columnas].astype(int)
columnas=['Número de SMS salientes','Total de llamadas salientes (miles)']
acceso_telefonia_movil[columnas]=acceso_telefonia_movil[columnas].astype(float)

In [62]:
# Creación de datasets limpios
df1 = pd.merge(penetracion_telefonia_movil, ingresos_telefonia_movil, on=["Año", "Trimestre"])
telefonia_movil=pd.merge(df1,acceso_telefonia_movil,on=["Año", "Trimestre"])
telefonia_movil.to_csv('./Datasets limpios/acceso_telefonia_movil.csv',index=False)

# Dataset de denuncias y reclamos

-- DF: Reclamos recibidos por delegación (año en curso)

In [63]:
reclamos_delegacion = pd.read_excel('./Datasets en bruto/Denuncias y reclamos/Denuncias_y_reclamos_202310.xlsx',sheet_name='Denuncias por mes',dtype=str)
reclamos_delegacion

,Año,Mes,Delegación,Reclamos
0,2023,1,Bahía Blanca - Buenos Aires,8
1,2023,1,Catamarca,26
2,2023,1,Chaco,33
3,2023,1,Comodoro Rivadavia - Chubut,12
4,2023,1,Córdoba,190
...,...,...,...,...
246,2023,9,SAU - Subdirección de Atención al Usuario,550
247,2023,9,Tierra del Fuego,7
248,2023,9,Tucumán,47
249,2023,9,Viedma - Río Negro,47


In [64]:
columnas=['Año','Mes','Reclamos']
reclamos_delegacion[columnas]=reclamos_delegacion[columnas].astype(int)

reclamos_delegacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Año         251 non-null    int32 
 1   Mes         251 non-null    int32 
 2   Delegación  251 non-null    object
 3   Reclamos    251 non-null    int32 
dtypes: int32(3), object(1)
memory usage: 5.0+ KB


-- DF : Reclamos mensuales recibidos por motivo de reclamo

In [65]:
denuncias_tipo = pd.read_excel('./Datasets en bruto/Denuncias y reclamos/Denuncias_y_reclamos_202310.xlsx',sheet_name='Denuncias por tipo',dtype=str)
denuncias_tipo=denuncias_tipo.astype(int)
denuncias_tipo

,Año,Mes,Problemas técnicos,Facturación,Gestiones administrativas,Otros,Total mensual
0,2023,1,1240,504,407,105,2256
1,2023,2,1140,468,393,91,2092
2,2023,3,1384,633,433,168,2618
3,2023,4,842,440,294,106,1682
4,2023,5,777,434,293,94,1598
5,2023,6,665,399,320,92,1476
6,2023,7,648,379,291,88,1406
7,2023,8,656,368,288,81,1393
8,2023,9,743,341,260,91,1435


-- DF : Reclamos por operador y motivo de reclamo

In [66]:
reclamos_operador = pd.read_excel('./Datasets en bruto/Denuncias y reclamos/Denuncias_y_reclamos_202310.xlsx',sheet_name='Reclamos_operador',dtype=str)
columnas=['Año','Mes','Cantidad']
reclamos_operador[columnas]=reclamos_operador[columnas].astype(int)
reclamos_operador

,Año,Mes,Motivo de Reclamo,Operador,Cantidad
0,2023,1,Facturación,AMX Argentina S.A. (Claro),16
1,2023,1,Facturación,DIRECTV Argentina S.A.L,4
2,2023,1,Facturación,Fibertel S.A. (TCI),8
3,2023,1,Facturación,Movistar - Telefonica Moviles Argentina S.A.,24
4,2023,1,Facturación,Otros,25
...,...,...,...,...,...
311,2023,9,Problemas Técnicos,Movistar - Telefonica Moviles Argentina S.A.,17
312,2023,9,Problemas Técnicos,Otros,58
313,2023,9,Problemas Técnicos,TELECENTRO S.A.,28
314,2023,9,Problemas Técnicos,Telecom Argentina S.A.,450


In [67]:
reclamos_operador.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316 entries, 0 to 315
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Año                316 non-null    int32 
 1   Mes                316 non-null    int32 
 2   Motivo de Reclamo  316 non-null    object
 3   Operador           316 non-null    object
 4   Cantidad           316 non-null    int32 
dtypes: int32(3), object(2)
memory usage: 8.8+ KB


-- DF : Reclamos mensuales por tipo de servicio

In [68]:
reclamos_servicios = pd.read_excel('./Datasets en bruto/Denuncias y reclamos/Denuncias_y_reclamos_202310.xlsx',sheet_name='Reclamos_servicio',dtype=str)
reclamos_servicios=reclamos_servicios.astype(int)
reclamos_servicios

,Año,Mes,Telefonía Fija,Internet Fijo,Comunicaciones Moviles,TV por cable,Total mensual
0,2023,1,1161,694,257,144,2256
1,2023,2,1032,658,257,145,2092
2,2023,3,1262,819,366,171,2618
3,2023,4,849,469,257,107,1682
4,2023,5,775,472,253,98,1598
5,2023,6,712,477,223,64,1476
6,2023,7,648,421,245,92,1406
7,2023,8,647,395,263,88,1393
8,2023,9,668,469,214,84,1435


In [69]:
# Creación de datasets limpios
reclamos_delegacion.to_csv('./Datasets limpios/reclamos_delegacion.csv',index=False)
denuncias_tipo.to_csv('./Datasets limpios/denuncias_tipo.csv',index=False)
reclamos_operador.to_csv('./Datasets limpios/reclamos_operador.csv',index=False)
reclamos_servicios.to_csv('./Datasets limpios/reclamos_servicios.csv',index=False)

# Dataset extra: Poblacion

In [70]:
poblacion_caba=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='02-CABA',dtype=str)
poblacion_caba=poblacion_caba.dropna().reset_index(drop=True)
poblacion_caba.columns=poblacion_caba.iloc[0]
poblacion_caba=poblacion_caba.iloc[1:,:2].reset_index(drop=True)
poblacion_caba.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_caba=poblacion_caba.astype(int)
poblacion_caba['Provincia']='Capital Federal'
poblacion_caba.head()

,Año,Poblacion,Provincia
0,2010,3028481,Capital Federal
1,2011,3033639,Capital Federal
2,2012,3038860,Capital Federal
3,2013,3044076,Capital Federal
4,2014,3049229,Capital Federal


In [71]:
poblacion_bsas=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='06-BUENOS AIRES',dtype=str)
poblacion_bsas=poblacion_bsas.dropna().reset_index(drop=True)
poblacion_bsas.columns=poblacion_bsas.iloc[0]
poblacion_bsas=poblacion_bsas.iloc[1:,:2].reset_index(drop=True)
poblacion_bsas.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_bsas=poblacion_bsas.astype(int)
poblacion_bsas['Provincia']='Buenos Aires'

poblacion_catamarca=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='10-CATAMARCA',dtype=str)
poblacion_catamarca=poblacion_catamarca.dropna().reset_index(drop=True)
poblacion_catamarca.columns=poblacion_catamarca.iloc[0]
poblacion_catamarca=poblacion_catamarca.iloc[1:,:2].reset_index(drop=True)
poblacion_catamarca.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_catamarca=poblacion_catamarca.astype(int)
poblacion_catamarca['Provincia']='Catamarca'

poblacion_cordoba=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='14-CÓRDOBA',dtype=str)
poblacion_cordoba=poblacion_cordoba.dropna().reset_index(drop=True)
poblacion_cordoba.columns=poblacion_cordoba.iloc[0]
poblacion_cordoba=poblacion_cordoba.iloc[1:,:2].reset_index(drop=True)
poblacion_cordoba.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_cordoba=poblacion_cordoba.astype(int)
poblacion_cordoba['Provincia']='Córdoba'

poblacion_corrientes=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='18-CORRIENTES',dtype=str)
poblacion_corrientes=poblacion_corrientes.dropna().reset_index(drop=True)
poblacion_corrientes.columns=poblacion_corrientes.iloc[0]
poblacion_corrientes=poblacion_corrientes.iloc[1:,:2].reset_index(drop=True)
poblacion_corrientes.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_corrientes=poblacion_corrientes.astype(int)
poblacion_corrientes['Provincia']='Corrientes'

poblacion_chaco=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='22-CHACO',dtype=str)
poblacion_chaco=poblacion_chaco.dropna().reset_index(drop=True)
poblacion_chaco.columns=poblacion_chaco.iloc[0]
poblacion_chaco=poblacion_chaco.iloc[1:,:2].reset_index(drop=True)
poblacion_chaco.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_chaco=poblacion_chaco.astype(int)
poblacion_chaco['Provincia']='Chaco'

poblacion_chubut=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='26-CHUBUT',dtype=str)
poblacion_chubut=poblacion_chubut.dropna().reset_index(drop=True)
poblacion_chubut.columns=poblacion_chubut.iloc[0]
poblacion_chubut=poblacion_chubut.iloc[1:,:2].reset_index(drop=True)
poblacion_chubut.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_chubut=poblacion_chubut.astype(int)
poblacion_chubut['Provincia']='Chubut'

poblacion_entre=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='30-ENTRE RÍOS',dtype=str)
poblacion_entre=poblacion_entre.dropna().reset_index(drop=True)
poblacion_entre.columns=poblacion_entre.iloc[0]
poblacion_entre=poblacion_entre.iloc[1:,:2].reset_index(drop=True)
poblacion_entre.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_entre=poblacion_entre.astype(int)
poblacion_entre['Provincia']='Entre Ríos'

poblacion_formosa=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='34-FORMOSA',dtype=str)
poblacion_formosa=poblacion_formosa.dropna().reset_index(drop=True)
poblacion_formosa.columns=poblacion_formosa.iloc[0]
poblacion_formosa=poblacion_formosa.iloc[1:,:2].reset_index(drop=True)
poblacion_formosa.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_formosa=poblacion_formosa.astype(int)
poblacion_formosa['Provincia']='Formosa'

poblacion_jujuy=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='38-JUJUY',dtype=str)
poblacion_jujuy=poblacion_jujuy.dropna().reset_index(drop=True)
poblacion_jujuy.columns=poblacion_jujuy.iloc[0]
poblacion_jujuy=poblacion_jujuy.iloc[1:,:2].reset_index(drop=True)
poblacion_jujuy.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_jujuy=poblacion_jujuy.astype(int)
poblacion_jujuy['Provincia']='Jujuy'

poblacion_pampa=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='42-LA PAMPA',dtype=str)
poblacion_pampa=poblacion_pampa.dropna().reset_index(drop=True)
poblacion_pampa.columns=poblacion_pampa.iloc[0]
poblacion_pampa=poblacion_pampa.iloc[1:,:2].reset_index(drop=True)
poblacion_pampa.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_pampa=poblacion_pampa.astype(int)
poblacion_pampa['Provincia']='La Pampa'

poblacion_rioja=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='46-LA RIOJA',dtype=str)
poblacion_rioja=poblacion_rioja.dropna().reset_index(drop=True)
poblacion_rioja.columns=poblacion_rioja.iloc[0]
poblacion_rioja=poblacion_rioja.iloc[1:,:2].reset_index(drop=True)
poblacion_rioja.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_rioja=poblacion_rioja.astype(int)
poblacion_rioja['Provincia']='La Rioja'

poblacion_mendoza=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='50-MENDOZA',dtype=str)
poblacion_mendoza=poblacion_mendoza.dropna().reset_index(drop=True)
poblacion_mendoza.columns=poblacion_mendoza.iloc[0]
poblacion_mendoza=poblacion_mendoza.iloc[1:,:2].reset_index(drop=True)
poblacion_mendoza.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_mendoza=poblacion_mendoza.astype(int)
poblacion_mendoza['Provincia']='Mendoza'

poblacion_misiones=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='54-MISIONES',dtype=str)
poblacion_misiones=poblacion_misiones.dropna().reset_index(drop=True)
poblacion_misiones.columns=poblacion_misiones.iloc[0]
poblacion_misiones=poblacion_misiones.iloc[1:,:2].reset_index(drop=True)
poblacion_misiones.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_misiones=poblacion_misiones.astype(int)
poblacion_misiones['Provincia']='Misiones'

poblacion_neuquen=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='58-NEUQUÉN',dtype=str)
poblacion_neuquen=poblacion_neuquen.dropna().reset_index(drop=True)
poblacion_neuquen.columns=poblacion_neuquen.iloc[0]
poblacion_neuquen=poblacion_neuquen.iloc[1:,:2].reset_index(drop=True)
poblacion_neuquen.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_neuquen=poblacion_neuquen.astype(int)
poblacion_neuquen['Provincia']='Neuquén'

poblacion_rio=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='62-RÍO NEGRO',dtype=str)
poblacion_rio=poblacion_rio.dropna().reset_index(drop=True)
poblacion_rio.columns=poblacion_rio.iloc[0]
poblacion_rio=poblacion_rio.iloc[1:,:2].reset_index(drop=True)
poblacion_rio.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_rio=poblacion_rio.astype(int)
poblacion_rio['Provincia']='Río Negro'

poblacion_salta=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='66-SALTA',dtype=str)
poblacion_salta=poblacion_salta.dropna().reset_index(drop=True)
poblacion_salta.columns=poblacion_salta.iloc[0]
poblacion_salta=poblacion_salta.iloc[1:,:2].reset_index(drop=True)
poblacion_salta.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_salta=poblacion_salta.astype(int)
poblacion_salta['Provincia']='Salta'

poblacion_juan=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='70-SAN JUAN',dtype=str)
poblacion_juan=poblacion_juan.dropna().reset_index(drop=True)
poblacion_juan.columns=poblacion_juan.iloc[0]
poblacion_juan=poblacion_juan.iloc[1:,:2].reset_index(drop=True)
poblacion_juan.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_juan=poblacion_juan.astype(int)
poblacion_juan['Provincia']='San Juan'

poblacion_luis=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='74-SAN LUIS',dtype=str)
poblacion_luis=poblacion_luis.dropna().reset_index(drop=True)
poblacion_luis.columns=poblacion_luis.iloc[0]
poblacion_luis=poblacion_luis.iloc[1:,:2].reset_index(drop=True)
poblacion_luis.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_luis=poblacion_luis.astype(int)
poblacion_luis['Provincia']='San Luis'

poblacion_santa=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='78-SANTA CRUZ',dtype=str)
poblacion_santa=poblacion_santa.dropna().reset_index(drop=True)
poblacion_santa.columns=poblacion_santa.iloc[0]
poblacion_santa=poblacion_santa.iloc[1:,:2].reset_index(drop=True)
poblacion_santa.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_santa=poblacion_santa.astype(int)
poblacion_santa['Provincia']='Santa Cruz'

poblacion_fe=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='82-SANTE FE',dtype=str)
poblacion_fe=poblacion_fe.dropna().reset_index(drop=True)
poblacion_fe.columns=poblacion_fe.iloc[0]
poblacion_fe=poblacion_fe.iloc[1:,:2].reset_index(drop=True)
poblacion_fe.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_fe=poblacion_fe.astype(int)
poblacion_fe['Provincia']='Santa Fe'

poblacion_estero=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='86-SANTIAGO DEL ESTERO',dtype=str)
poblacion_estero=poblacion_estero.dropna().reset_index(drop=True)
poblacion_estero.columns=poblacion_estero.iloc[0]
poblacion_estero=poblacion_estero.iloc[1:,:2].reset_index(drop=True)
poblacion_estero.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_estero=poblacion_estero.astype(int)
poblacion_estero['Provincia']='Santiago Del Estero'

poblacion_tuc=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='90-TUCUMÁN',dtype=str)
poblacion_tuc=poblacion_tuc.dropna().reset_index(drop=True)
poblacion_tuc.columns=poblacion_tuc.iloc[0]
poblacion_tuc=poblacion_tuc.iloc[1:,:2].reset_index(drop=True)
poblacion_tuc.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_tuc=poblacion_tuc.astype(int)
poblacion_tuc['Provincia']='Tucumán'

poblacion_tierra=pd.read_excel('./Dataset extra - Poblacion/c1_proyecciones_prov_2010_2040.xlsm',sheet_name='94-TIERRA DEL FUEGO',dtype=str)
poblacion_tierra=poblacion_tierra.dropna().reset_index(drop=True)
poblacion_tierra.columns=poblacion_tierra.iloc[0]
poblacion_tierra=poblacion_tierra.iloc[1:,:2].reset_index(drop=True)
poblacion_tierra.rename(columns={'Ambos sexos': 'Poblacion'}, inplace=True)
poblacion_tierra=poblacion_tierra.astype(int)
poblacion_tierra['Provincia']='Tierra Del Fuego'

In [72]:
poblacion_nac=pd.concat([poblacion_caba,poblacion_bsas,poblacion_catamarca,poblacion_cordoba,poblacion_corrientes,poblacion_chaco,poblacion_chubut,poblacion_entre,poblacion_formosa,poblacion_jujuy,poblacion_pampa,poblacion_rioja,poblacion_mendoza,poblacion_misiones,poblacion_neuquen,poblacion_rio,poblacion_salta,poblacion_juan,poblacion_luis,poblacion_santa,poblacion_fe,poblacion_estero,poblacion_tuc,poblacion_tierra])
poblacion_nac

,Año,Poblacion,Provincia
0,2010,3028481,Capital Federal
1,2011,3033639,Capital Federal
2,2012,3038860,Capital Federal
3,2013,3044076,Capital Federal
4,2014,3049229,Capital Federal
...,...,...,...
26,2036,241593,Tierra Del Fuego
27,2037,245734,Tierra Del Fuego
28,2038,249853,Tierra Del Fuego
29,2039,253948,Tierra Del Fuego


In [73]:
poblacion_nac.to_csv('./Datasets limpios/poblacion_nac.csv',index=False)